In [7]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import logging
import time
import sys

PROJECT_PATH = "/Users/shawn/Documents/personal/rsi_divergence_detector"
sys.path.append(PROJECT_PATH)
# Load the training data
# Replace 'training_data.csv' with your actual data file or DataFrame
df_total = pd.read_pickle(f'{PROJECT_PATH}/data/training_data.pickle')
divergence_data = pd.read_pickle(f"{PROJECT_PATH}/data/divergence_data.pickle")

In [17]:
# Test dataframes sliced
start_time = "2024-11-11"
df = df_total[df_total.timeframe == '15m']
df = df.loc[df.index >= start_time]
# df = df.loc[df.index <= '2024-11-27 00:00:00']
# df = df.loc[df.index >= '2024-11-22 12:00:00']
# df = df.loc[df.index <= '2024-11-22 16:15:00']


dd_filter = divergence_data.copy()
for key, value in dd_filter.items():
    value = value.loc[value.index >= start_time]
    value = value.sort_values('end_datetime').sort_index()
    dd_filter[key] = value
    print(f"{key} filtered data length = {len(value)}")


5m filtered data length = 251
15m filtered data length = 85
1h filtered data length = 28
4h filtered data length = 9
1d filtered data length = 4


## Adding different timeline divergence data

In [42]:
dd_filter['5m'][110:118]

,end_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,TP,SL,label,TP_percent,SL_percent,TP_/_SL,profit,div_5m,div_15m,div_1h,div_4h,div_1d
start_datetime,,,,,,,,,,,,,,,,,,,
2024-11-22 03:15:00,2024-11-22 07:40:00,2024-11-22 07:50:00,99324.00,2024-11-22 01:45:00,Bearish Divergence,-220.94,-14.09,98816.43,99419.99,True,0.51,0.10,5.29,507.57,False,True,True,True,True
2024-11-22 03:15:00,2024-11-22 17:15:00,2024-11-22 17:25:00,99085.82,2024-11-22 01:45:00,Bearish Divergence,-283.56,-12.86,98679.86,99199.00,False,0.41,0.11,3.59,-113.18,False,True,True,True,True
2024-11-22 13:25:00,2024-11-22 14:55:00,2024-11-22 15:05:00,97487.97,2024-11-22 12:00:00,Bullish Divergence,1102.23,26.32,97819.98,97122.11,True,0.34,0.38,0.91,332.01,False,True,NaN,True,True
2024-11-22 17:15:00,2024-11-22 19:05:00,2024-11-22 19:15:00,99212.13,2024-11-22 14:55:00,Bearish Divergence,165.32,-0.45,98520.34,99384.62,False,0.70,0.17,4.01,-172.49,False,NaN,NaN,True,True
2024-11-23 16:25:00,2024-11-23 17:05:00,2024-11-23 17:15:00,97517.84,2024-11-23 15:00:00,Bullish Divergence,532.70,20.01,97887.53,97336.06,True,0.38,0.19,2.03,369.69,False,True,NaN,True,True
2024-11-23 16:25:00,2024-11-23 19:35:00,2024-11-23 19:45:00,97412.20,2024-11-23 15:00:00,Bullish Divergence,-6.02,2.81,97903.56,97362.00,False,0.50,0.05,9.79,-50.20,False,True,NaN,True,True
2024-11-23 16:25:00,2024-11-23 20:10:00,2024-11-23 20:20:00,97341.22,2024-11-23 15:00:00,Bullish Divergence,221.00,16.12,97872.04,97311.00,False,0.55,0.03,17.57,-30.22,False,True,NaN,True,True
2024-11-23 16:25:00,2024-11-23 20:25:00,2024-11-23 20:35:00,97572.00,2024-11-23 15:00:00,Bullish Divergence,399.24,21.30,97763.89,97136.00,True,0.20,0.45,0.44,191.89,False,True,NaN,True,True


In [43]:
dd_filter['15m'][30:36]

,end_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,TP,SL,label,...,SL_percent,position,TP_vs_SL,TP_/_SL,profit,div_15m,div_5m,div_1h,div_4h,div_1d
start_datetime,,,,,,,,,,,,,,,,,,,,,
2024-11-21 04:00:00,2024-11-22 07:30:00,2024-11-22 08:00:00,99173.70,2024-11-21 02:30:00,Bearish Divergence,-557.13,-17.56,97364.83,99419.99,True,...,0.25,True,7.344456,7.34,1808.87,False,True,True,True,True
2024-11-22 13:15:00,2024-11-22 14:45:00,2024-11-22 15:15:00,97951.99,2024-11-22 07:30:00,Bullish Divergence,933.42,19.24,97999.90,97122.11,True,...,0.85,False,0.057731,0.06,47.91,False,True,NaN,True,True
2024-11-23 17:00:00,2024-11-24 11:45:00,2024-11-24 12:15:00,97360.64,2024-11-23 14:45:00,Bullish Divergence,-62.67,10.61,97829.31,97239.29,False,...,0.12,False,3.862150,3.86,-121.35,False,True,NaN,True,True
2024-11-23 17:00:00,2024-11-24 11:15:00,2024-11-24 11:45:00,97496.97,2024-11-23 14:45:00,Bullish Divergence,-801.74,-9.52,97851.38,97275.00,False,...,0.23,False,1.596660,1.60,-221.97,False,True,NaN,True,True
2024-11-23 17:00:00,2024-11-23 20:15:00,2024-11-23 20:45:00,97691.54,2024-11-23 14:45:00,Bullish Divergence,694.38,22.00,97765.48,97136.00,True,...,0.57,False,0.133093,0.13,73.94,False,True,NaN,True,True
2024-11-23 17:00:00,2024-11-23 19:30:00,2024-11-23 20:00:00,97504.00,2024-11-23 14:45:00,Bullish Divergence,291.33,14.44,97905.15,97362.00,False,...,0.15,False,2.824977,2.82,-142.00,False,True,NaN,True,True


In [ ]:
# Assuming dd_filter is a dictionary where keys are timeframes ('5m', '15m', etc.)
# and values are the respective DataFrames.
timeframe_to_minutes = {
    '5m': 5,
    '15m': 15,
    '1h': 60,
    '4h': 240,
    '1d': 1440  # 하루는 1440분
}
# Define the timeframes for which to check divergence
for timeframe_key in dd_filter.keys():
    for compare_key in dd_filter.keys():
        if timeframe_key != compare_key:
            dd_filter[timeframe_key][f"div_{compare_key}"] = False

# Iterate over all timeframes to compare divergences
for base_timeframe, base_df in dd_filter.items():
    for compare_timeframe, compare_df in dd_filter.items():
        if base_timeframe == compare_timeframe:
            continue  # 동일한 시간봉은 비교하지 않음

        # 비교 시간봉의 간격(분)을 가져옴
        compare_interval = pd.to_timedelta(timeframe_to_minutes[compare_timeframe], unit='m')

        # 각 base_df의 row에 대해 비교
        for base_index, _ in base_df.iterrows():
            base_start = base_index

            # 비교 시간봉의 모든 행에 대해 시작 시간 범위 체크
            for compare_index, _ in compare_df.iterrows():
                compare_start = compare_index

                # 기준 시간(base_start)이 비교 시간 범위(compare_start ~ compare_start + compare_interval)에 있는지 확인
                if compare_start <= base_start < (compare_start + compare_interval):
                    # 조건을 만족하면 서로의 다이버전스 컬럼을 True로 설정
                    base_df.at[base_index, f"div_{compare_timeframe}"] = True
                    compare_df.at[compare_index, f"div_{base_timeframe}"] = True


KeyError: 'start_datetime'

In [40]:
compare_row

end_datetime              2024-11-12 07:00:00
entry_datetime            2024-11-12 07:30:00
entry_price                          89347.98
previous_peak_datetime    2024-11-10 21:00:00
divergence                 Bearish Divergence
price_change                         -1059.99
rsi_change                             -18.08
TP                                   85536.79
SL                                    89903.0
label                                    True
TP_percent                               4.27
SL_percent                               0.62
position                                 True
TP_vs_SL                             6.866762
TP_/_SL                                  6.87
profit                                3811.19
div_15m                                 False
Name: 2024-11-11 16:00:00, dtype: object

## Analyze

In [84]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# 서비스 계정 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(f'{PROJECT_PATH}/calcium-ember-444319-n7-3b60cf57e696.json', scope)
client = gspread.authorize(credentials)
# Google Service Account Credentials JSON 파일 경로
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1uJy2-CV63Pywc2GJJGRP6fSHXmPuTTybS8bMIWhH4Qc/edit?gid=0#gid=0")


Uploading to Google Sheets

In [ ]:
divergence_df2 = divergence_df
sheets = {}

for timeframe, ddf in divergence_df.items():
    ddf['TP_percent'] = 100 * (ddf['TP'] - ddf['entry_price']) * np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1) / ddf['entry_price'] 
    ddf['SL_percent'] = 100 * (ddf['entry_price'] - ddf['SL']) * np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1) / ddf['entry_price']
    ddf['TP_/_SL'] = ddf['TP_percent'] / ddf['SL_percent']
    is_bullish = np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1)
    ddf['profit'] = np.where(
        ddf['label'],
        is_bullish * (ddf['TP'] - ddf['entry_price']),
        -is_bullish * (ddf['entry_price'] - ddf['SL'])
    )
    if 'future_return' in ddf.columns:
        ddf = ddf.drop(columns=['future_return'])
    ddf[['price_change', 'rsi_change', 'TP', 'SL', 'TP_percent', 'SL_percent', 'TP_/_SL', 'profit']] = ddf[['price_change', 'rsi_change', 'TP', 'SL', 'TP_percent', 'SL_percent', 'TP_/_SL', 'profit']].round(2)
    ddf2 = ddf.copy()
    ddf2['start_datetime'] = ddf2.index
    # start_datetime 열을 가장 왼쪽으로 이동
    ddf2 = ddf2[['start_datetime'] + [col for col in ddf2.columns if col != 'start_datetime']]

    ddf2 = ddf2.astype(str)

    divergence_df2[timeframe] = ddf
    sheets[timeframe] = ddf2

pd.to_pickle(divergence_df2, f"{PROJECT_PATH}/data/divergence_data3.pickle")

for sheet_name, df in sheets.items():
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")
    worksheet.clear()  # 기존 데이터 삭제
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    
# TODO: in first sheet, add total profits

/var/folders/ns/shg9ht0545sbbr41sk4q_zgr0000gn/T/ipykernel_93962/804550579.py:35: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


Analyzing True / False counts and profits

In [ ]:
from collections import Counter

for timeframe in ['5m', '15m', '1h', '4h', '1d']:
    print(f"==========Analyzing {timeframe} profit and counts of position==========")
    ddf = divergence_df[timeframe]
    c = Counter(ddf.label)
    profit = 0
    for _, row in ddf.iterrows():
        is_bullish = 1 if row.divergence == "Bullish Divergence" else -1
        if row.label:
            profit += is_bullish * (row.TP - row.entry_price)
        else:
            profit -= is_bullish * (row.entry_price - row.SL)
    
    print(f"{timeframe} Total True / False count = {c}")
    print(f"{timeframe} Total profit = {profit}")
    print('\n')
    
    ddf_11 = ddf.loc[ddf.index >= '2024-11-01']
    c_11 = Counter(ddf_11.label)
    profit_11 = 0
    for _, row in ddf_11.iterrows():
        is_bullish = 1 if row.divergence == "Bullish Divergence" else -1
        if row.label:
            profit_11 += is_bullish * (row.TP - row.entry_price)
        else:
            profit_11 -= is_bullish * (row.entry_price - row.SL)
    print(f"{timeframe} after 24.11 True / False count = {c_11}")
    print(f"{timeframe} after 24.11 profit = {profit_11}")
    print('\n\n')



==========Analyzing 5m profit and counts of position==========
5m after 24.11 True / False count = Counter({False: 256, True: 95})
5m after 24.11 profit = 10430.97455999993



==========Analyzing 15m profit and counts of position==========
15m after 24.11 True / False count = Counter({False: 92, True: 41})
15m after 24.11 profit = 14577.046020000038



==========Analyzing 1h profit and counts of position==========
1h after 24.11 True / False count = Counter({False: 32, True: 7})
1h after 24.11 profit = -15634.050520000063



==========Analyzing 4h profit and counts of position==========
4h after 24.11 True / False count = Counter({False: 8, True: 2})
4h after 24.11 profit = -10419.922420000003



==========Analyzing 1d profit and counts of position==========
1d after 24.11 True / False count = Counter({False: 4})
1d after 24.11 profit = -8177.369999999995



